In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

## 总结

+ 批归一化：对每个特征进行归一化。  `input_x.mean(dim=(0, 1), keepdim=True)`
+ 层归一化：对每个样本、每层归一化，层在NLP中可以理解为 时间维度。`input_x.mean(dim=(2), keepdim=True)`
+ 实例归一化：对每个样本、每个通道（特征）做归一化,通常用在分格迁移中。`input_x.mean(dim=(1), keepdim=True)`
+ 群归一化：对每个样本、每个分组进行归一化。
+ 权重归一化：对权重进行归一化，再缩放的操作。

统计量（均值、标准差）的维度：
+ 批归一化
    + [batch_size, time_steps, features] -> [features]
    + [batch_size, channel, height, width] -> [channel]
+ 层归一化：
    + [batch_size, time_steps, features] -> [batch_size, time_steps]
    + [batch_size, channel, height, width] -> [batch_size, height, width]
+ 实例归一化：
    + [batch_size, groups, time_steps, features//groups] -> [batch_size, groups]
    + [batch_size, groups, channel//groups, height, width] -> [batch_size, groups]

当然计算统计量都会keepdim，


## BatchNorm

计算**整个mini-batch各个特征统计量**：
+ 期望 $E[x]$
+ 标准差: $Var[x]$, 这里计算的是有偏估计，即除 $n$，而不是除$n-1$

$$
y=\frac{x-\mathrm{E}[x]}{\sqrt{\operatorname{Var}[x]+\epsilon}} * \gamma+\beta
$$

Pytorch中，期望方差的计算并不是当前mini-batch，而是最近多个batch的加权移动平均，所以有一个参数 momentum=0.1:
$$
\hat{x}_{\text {new }}=(1-\text { momentum }) \times \hat{x}+\text { momentum } \times x_{t}
$$

很少用到NLP任务中，因为这类任务一个batch中不同样本的序列长度可能不一样，涉及到填充补0的操作，如果采用batch_norm，不同的batch中计算得到结果就不一致了，归一化之后的缩放平移操作就无法统一了。

In [90]:
batch_size = 2
time_steps = 2
input_features = 4
groups_num = 2

In [77]:
inputx = torch.normal(-10, 10, (batch_size, time_steps, input_features))

In [78]:
# torch 官方api； batch_noram
batch_norm_op = nn.BatchNorm1d(num_features=input_features, affine=False)
bn_y = batch_norm_op(inputx.transpose(-1, -2)).transpose(-1, -2)

In [79]:
# 手写 batch_noram

# dim=(0, 1) 表示的是 在不同样本间间、在不同时间维度间 的数值求均值，也就是求的特征维度的均值，可以类比二维矩阵求均值dim=0是各个特征的均值
# 均值
bn_mean = inputx.mean(dim=(0, 1), keepdim=True) # shape:[4]
# 标准差：计算的是有偏估计
bn_std = inputx.std(dim=(0, 1), unbiased=False, keepdim=True) # shape:[4]

my_bn_y = (inputx - bn_mean) / (bn_std + 1e-9)

In [80]:
print(torch.allclose(bn_y, my_bn_y))

bn_y, my_bn_y

True


(tensor([[[-0.4297,  1.5990,  0.7366, -1.7262],
          [-0.9797, -1.0223, -1.3768,  0.4928]],
 
         [[-0.2602,  0.0521, -0.5036,  0.7077],
          [ 1.6697, -0.6288,  1.1438,  0.5257]]]),
 tensor([[[-0.4297,  1.5990,  0.7366, -1.7262],
          [-0.9797, -1.0223, -1.3768,  0.4928]],
 
         [[-0.2602,  0.0521, -0.5036,  0.7077],
          [ 1.6697, -0.6288,  1.1438,  0.5257]]]))

## Layer Norm

计算的是**每一样本，每一层**的归一化， 通常用在NLP任务中。在nlp任务中，每一层即每一个时刻的lstm单元。所以计算均值、标准差的时候计算的维度dim=2

In [81]:
# 官方 api

# 不做平移和缩放
layer_norm_op = nn.LayerNorm(input_features, elementwise_affine=False)
ln_y = layer_norm_op(inputx) # 只需要保证 输入的维度中batch在第一维就可以

In [82]:
# 手写 layer_norm

ln_mean = inputx.mean(dim=2, keepdim=True)

ln_std = inputx.std(dim=2, keepdim=True, unbiased=False)

my_ln_y = (inputx - ln_mean) / (ln_std + 1e-5)

In [83]:
print(torch.allclose(ln_y, my_ln_y))
ln_y, my_ln_y

True


(tensor([[[-0.9096, -0.0775,  1.6510, -0.6640],
          [-0.9630, -0.6483, -0.0173,  1.6285]],
 
         [[-0.6639, -1.0249,  0.1086,  1.5802],
          [ 1.0997, -1.6217,  0.3717,  0.1503]]]),
 tensor([[[-0.9095, -0.0775,  1.6510, -0.6640],
          [-0.9630, -0.6483, -0.0173,  1.6285]],
 
         [[-0.6639, -1.0249,  0.1086,  1.5802],
          [ 1.0997, -1.6217,  0.3717,  0.1503]]]))

## 实例归一化 Instance Norm

对**每个样本，每个通道**进行归一化，通常用在风格迁移上面。实例归一化时一般是不需要进行重缩放平移操作的。

理解：通过归一化，把不同时刻共有的信息给抹除掉了，共有的信息即分格！比如说音频信号，不同时刻共有的信息即声音的音调，把音调给抹除掉。  

In [84]:
# 官方api
ins_norm_op = torch.nn.InstanceNorm1d(input_features, affine=False)
ins_y = ins_norm_op(inputx.transpose(-1, -2)).transpose(-1, -2)

In [85]:
# 手写实例归一化

ins_norm_mean = inputx.mean(dim=1, keepdim=True)
ins_norm_std = inputx.std(dim=1, keepdim=True, unbiased=False)

my_ins_norm_y = (inputx - ins_norm_mean) / (ins_norm_std + 1e-5)

In [86]:
print(torch.allclose(ins_y, my_ins_norm_y))
ins_y, my_ins_norm_y

True


(tensor([[[ 1.0000,  1.0000,  1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  1.0000]],
 
         [[-1.0000,  1.0000, -1.0000,  1.0000],
          [ 1.0000, -1.0000,  1.0000, -1.0000]]]),
 tensor([[[ 1.0000,  1.0000,  1.0000, -1.0000],
          [-1.0000, -1.0000, -1.0000,  1.0000]],
 
         [[-1.0000,  1.0000, -1.0000,  1.0000],
          [ 1.0000, -1.0000,  1.0000, -1.0000]]]))

## 群归一化

对**每个样本 每个group**最归一化。这里的group 类似群组卷积，先将通道（特征）进行一个群组划分，然后对每个样本，每个group进行归一化，和batch_norm的区别在于，群归一化是对每个样本做的归一化，而batch_norm是对所有样本一起的

In [91]:
# 官方 api
group_norm_op = nn.GroupNorm(num_groups=groups_num, num_channels=input_features)
group_norm_y = group_norm_op(inputx.transpose(-1, -2)).transpose(-1, -2)

In [94]:
# 手写 群归一化
group_inputx = torch.split(inputx, split_size_or_sections=input_features//groups_num, dim=2)
results = []

# for 循环是选中每个group，训练里面对每个样本
for g_i in group_inputx:
    # [g_i batch_size, time_steps, input_features//groups_num]
    gn_mean = g_i.mean(dim=(1, 2), keepdim=True) # 0 维是样本，
    gn_std = g_i.std(dim=(1, 2), keepdim=True, unbiased=False)
    gn_res = (g_i - gn_mean) / (gn_std + 1e-5)

    results.append(gn_res)

my_gn_y = torch.cat(results, dim=-1)

In [95]:
torch.allclose(group_norm_y, my_gn_y)

True

## 权重归一化

前面的归一化均是对输入进行归一化，而此处的权重归一化是对 模型的权重进行归一化。
把权重 的方向(单位向量)和大小解耦开来：
$$
\mathbf{w}=g \frac{\mathbf{v}}{\|\mathbf{v}\|}
$$
原先的参数是 $v$，通过$\frac{\mathbf{v}}{\|\mathbf{v}\|}$ 得到 参数的方向向量，然后再引入一个新的参数$g$，表示大小，g是可学习的参数。此时新的权重即为 $w$：保留了原先权重的方向，引入了一个可学习的大小（幅度）

